# **1. Importación de *modules***

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# **2. Importación del dataset *properatti_tp2.csv***

In [2]:
data = pd.read_csv("../data/properatti_tp2.csv", sep = ",", low_memory=False) 
data.head(3)
data.shape

,property_type,state_name,place_with_parent_names,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,rooms,description,...,s.u.m.,amenities,baulera,gimnasio,subte-linea-d,subte-linea-b,subte-linea-a,subte-linea-h,subte-linea-e,price_state
0,PH,Capital Federal,|Argentina|Capital Federal|Mataderos|,62000.0,55.0,40.0,1127.272727,1550.000000,2.0,"2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO,...",...,False,False,False,False,False,False,False,False,False,134.214298
1,apartment,Bs.As. G.B.A. Zona Sur,|Argentina|Bs.As. G.B.A. Zona Sur|La Plata|,150000.0,NaN,NaN,NaN,NaN,3.0,VENTA DE DEPARTAMENTO EN DÉCIMO PISO AL FRENTE...,...,False,False,False,False,False,False,False,False,False,226.743900
2,apartment,Capital Federal,|Argentina|Capital Federal|Mataderos|,72000.0,55.0,55.0,1309.090909,1309.090909,2.0,2 AMBIENTES 3ER PISO LATERAL LIVING COMEDOR AM...,...,False,False,False,False,False,False,False,False,False,134.214298


(56824, 30)

# **3. Tratamiento de los datos**